In [1]:
# Import Libraries
from os import listdir
from os.path import isfile, join
import cv2

In [2]:
# Load our Neural Transfer Models
model_file_path = './models/'
model_file_paths = [f for f in listdir(model_file_path) if isfile(join(model_file_path, f))]

In [3]:
# Load our Test Image
img = cv2.imread('./images/Yuki.jpg')

In [4]:
# Load our Transfer Style Model
model = ('la_muse.t7')

In [5]:
# Loop and Apply each Model Style our Test Image
for i in model_file_paths:
    style = cv2.imread('./art/' + str(model)[:-3] + '.jpg')
    # Load our Neural Transfer Style Model
    neural_style_model = cv2.dnn.readNetFromTorch(model_file_path + model)
    
    # Resize it to fix height. You can change it as you want
    height, width = int(img.shape[0]), int(img.shape[1])
    new_width = int((640/height) * width)
    resized_image = cv2.resize(img,
                              (new_width, 640),
                              interpolation = cv2.INTER_AREA)
    
    # Create our blob from the image
    # Then perform a forward pass run of the network
    # The Mean values for the ImageNet training set are R=103.93, G=116.77, B=123.68
    inp_blob = cv2.dnn.blobFromImage(resized_image,
                                     1.0,
                                    (new_width,640),
                                    (103.93, 116.77, 123.68),
                                    swapRB = False,
                                    crop = False)
    
    neural_style_model.setInput(inp_blob)
    output = neural_style_model.forward()
      
    # Reshape the output Tensor, 
    # add back the mean substruction,
    # re-order the channels
    output = output.reshape(3,output.shape[2],output.shape[3])
    output[0] += 103.93
    output[1] += 116.77
    output[2] += 123.68
    output /= 255
    output = output.transpose(1, 2, 0)
    
    # Display:
    # 1. Original/Test Image
    # 2. The Style of the Neural Transfer
    # 3. Our result from them
    cv2.imshow('Original', img)
    cv2.imshow('Style', style)
    cv2.imshow('Neual Transfer Style', output)
    cv2.waitKey(0)
        
    # Close everything
    if cv2.waitKey(0) & 0xFF ==27:
        break
    
    
# Destroy All Windows
cv2.destroyAllWindows()

